# Build a Crew to Tailor Job Applications

## Setting up the LLM and the agent tools

In [199]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from IPython.display import Markdown, display

from dotenv import load_dotenv
import os

# - Import libraries, APIs and LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew

# ## crewAI Tools
from crewai_tools import (
  FileReadTool,
  SeleniumScrapingTool,
  MDXSearchTool,
  SerperDevTool
)

load_dotenv()  # Load environment variables from the .env file

gemini_llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',
  verbose=True,
  temperature=0.5,
  goggle_api_key=os.getenv('GOOGLE_API_KEY'))               

search_tool = SerperDevTool()
scrape_tool = SeleniumScrapingTool()
read_resume = FileReadTool(file_path='./data/resume_data.md')
semantic_search_resume = MDXSearchTool(mdx='./data/resume_data.md')


# - Uncomment and run the cell below if you wish to view `resume_data.md` in the notebook.
# display(Markdown("./data/resume_data.md"))

Inserting batches in chromadb:   0%|          | 0/1 [00:01<?, ?it/s]


## Creating Agents

In [200]:
job_posting_url='https://apply.workable.com/huggingface/j/539C305C61/'#'https://moises.bamboohr.com/careers/43'
custom_scrape_tool = SeleniumScrapingTool(website_url=job_posting_url, css_element='body')
read_job_post = FileReadTool(file_path='./data/job_post.md')

job_researcher = Agent(    
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",         
    tools = [
        read_job_post,
        # custom_scrape_tool
    ],    
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=gemini_llm
)

# company_researcher = Agent(
#     role="Tech Company Researcher",
#     goal="Perform thorough analysis on tech companies, to find out about what they do and their finantials",
#     tools = [scrape_tool, search_tool],
#     verbose=True,
#     backstory=(
#         "As a Company Researcher, your prowess in navigating and extracting critical "
#         "information from a company is unmatched. Your skills help identify the mission and core values of a company,"
#         "its main products, clients and competitors, "
#         "all which provides tremendous insights for effective application tailoring and interview preparation."
#     )
# )

profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources, to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=gemini_llm
)

# Writer
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=gemini_llm
)

# interview_preparer = Agent(
#     role="Engineering Interview Preparer",
#     goal="Create interview questions and talking points "
#          "based on the resume and job requirements",
#     tools = [scrape_tool, search_tool,
#              read_resume, semantic_search_resume],
#     verbose=True,
#     backstory=(
#         "Your role is crucial in anticipating the dynamics of "
#         "interviews. With your ability to formulate key questions "
#         "and talking points, you prepare candidates for success, "
#         "ensuring they can confidently address all aspects of the "
#         "job they are applying for."
#     )
# )

## Creating Tasks

In [201]:
# Task for Job Researcher Agent: Extract Job Requirements
job_research_task = Task(
    description=(
        "Using the provided tools, analyze the job posting "
        "to extract the required key skills, experiences, and qualifications. "
        "Include personal treats and values that may be relevant. "
    ),
    expected_output=(
        "A structured list of job requirements, "
        "including necessary skills, qualifications, and experiences."
    ),
    output_file="output/job_requirements.md",
    agent=job_researcher,
    async_execution=True,
)

# Task for Company Researcher Agent: Extract Company Details
# company_research_task = Task(
#     description=(
#         "Use the tools to perform a comprehensive research on the company ({company_name} - {company_url}). "
#         "Find out about its mission, core values, main products and clients. "
#         "Investigate about its financial situation, valuation, funding, investors "
#         "and a potential liquidity event (acquisition, IPO...)"
#     ),
#     expected_output=(
#         "A detailed report about the company including its mission, core values, main products and clients. "
#         "It should also include a complete financial report covering valuation, funding, investors and potential liquidity events"
#     ),
#     output_file="data/company_profile.md",
#     agent=company_researcher,
#     async_execution=True
# )

# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile using ONLY the candidate resume data "
        "and any website provided in it (e.g., Github page). "
        "Utilize tools to extract and synthesize information from these sources. "
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests and "
        "communication style."
    ),
    output_file="output/candidate_profile.md",
    agent=profiler,
    async_execution=True
)


# - You can pass a list of tasks as `context` to a task.
# - The task then takes into account the output of those tasks in its execution.
# - The task will not run until it has the output(s) from those tasks.


# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile, job requirements and company info obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "including the initial summary, work experience, skills, "
        "and education. All to better reflect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="output/tailored_resume.md",
    context=[job_research_task, profile_task],
    agent=resume_strategist
)


# Task for Interview Preparer Agent: Develop Interview Materials
# interview_preparation_task = Task(
#     description=(
#         "Create a set of potential interview questions and talking "
#         "points based on the tailored resume and job requirements. "
#         "Utilize tools to generate relevant questions and discussion "
#         "points. Make sure to use these question and talking points to "
#         "help the candiadte highlight the main points of the resume "
#         "and how it matches the job posting."
#     ),
#     expected_output=(
#         "A document containing key questions and talking points "
#         "that the candidate should prepare for the initial interview."
#     ),
#     output_file="interview_materials.md",
#     context=[research_task, profile_task, resume_strategy_task],
#     agent=interview_preparer
# )

## Creating the Crew

In [202]:
job_application_crew = Crew(
    agents=[
        job_researcher,
        # profiler,
        # resume_strategist,
        # interview_preparer
        ],

    tasks=[
        job_research_task,
        # profile_task,
        # resume_strategy_task,
        # interview_preparation_task
        ],

    verbose=True
)

2024-10-18 15:55:45,979 - 8376646336 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

In [203]:
# - Set the inputs for the execution of the crew.
job_application_inputs = {
    'job_posting_url': job_posting_url,
    'company_name': 'music.ai',
    'company_url': 'https://music.ai/'    
}

### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Using the provided tools, analyze the job posting to extract the required key skills, experiences, and qualifications. Include personal treats and values that may be relevant. 
 [DEBUG]: == [Tech Job Researcher] Task output: 




> Entering new CrewAgentExecutor chain...


Thought: I need to read the job posting to extract the required information.

Action: Read a file's content
Action Input: {"file_path": "./data/job_post.md"} 

Description
Here at Hugging Face, we’re on a journey to advance good Machine Learning and make it more accessible. Along the way, we contribute to the development of technology for the better.

We have built the fastest-growing, open-source, library of pre-trained models in the world. With more than 1 Million+ models and 320K+ stars on GitHub, over 15.000 companies are using HF technology in production, including leading AI organizations such as Google, Elastic, Salesforce, Grammarly and NASA.

About the role

As a frontend engineer, you will work with core web technologies and python to build complex UI components to empower users to develop powerful applications with minimal code. You will respond quickly to the constantly changing needs of our user base to enable new and more exciting use cases.

You'll help maintain a popula

2024-10-18 15:55:58,941 - 20754821120 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Thought: I need to read the job posting to extract the required information.

Action: Read a file's content
Action Input: {"file_path": "./data/job_post.md"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





2024-10-18 15:56:01,934 - 20754821120 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
2024-10-18 15:56:04,155 - 20754821120 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
2024-10-18 15:56:08,384 - 20754821120 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
2024-10-18 15:56:16,616 - 20754821120 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as i

Thought: I need to read the job posting to extract the required information.

Action: Read a file's content
Action Input: {"file_path": "./data/job_post.md"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to read the job posting to extract the required information.

Action: Read a file's content
Action Input: {"file_path": "./data/job_post.md"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to read the job posting to extract the required information.

Action: Read a file's content
Action Input: {"file_path": "./data/job_post.md"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to read the job posting to extract the required information.

Action: Read a file's content
Action Input: {"file_path": "./data/job_post.md"} 

I tried reusing the same input, I must stop u

In [198]:
os.remove('./output/job_requirements.md')